In [241]:
import torch
from torch.utils.data import IterableDataset, DataLoader, Subset
from datetime import datetime as dt, timedelta
import pandas as pd
import os
import random
import numpy as np
import torch.nn as nn
from pandas import DataFrame as df
import mplfinance as mpf

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

seed = 42  # choose any seed you prefer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

cuda


In [242]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = f'../csvfiles/{item}'
        self.item = item
        self.timespan = timespan
        start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [1, 4]  # Selecting open and close prices
        self.filenames = self.get_filenames()

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def get_filenames(self):
        filenames = []
        for date in self.dates:
            filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
            if os.path.exists(filename):
                filenames.append(filename)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        return torch.tensor(df.values, dtype=torch.float32)  # Return open and close prices


def sliding_window_pct(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 100 + 1):  # Create windows of 100 rows each
            window = tensor[i:i+100]
            # Calculate the percentage change for each pair in the window
            pct_change = (window[:, 1] - window[:, 0]) / window[:, 0] * 100
            windows.append(pct_change)
    # Only keep the first 100 windows if there are more than 100
    return torch.stack(windows)




In [243]:
# Create the dataset
train_dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')
train_loader = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)


test_dataset = PriceDataset('BTCUSDT', '1m', '2023-05-01', '2023-05-15')
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)



In [244]:
class PriceChangePrediction(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, output_dim=10, num_layers=4):
        super(PriceChangePrediction, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout = 0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out
    
class PriceDirectionPrediction(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, output_dim=10, num_layers=4):
        super(PriceDirectionPrediction, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout = 0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


In [245]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch in train_loader:
        batch = batch.unsqueeze(-1).to(device)  # Adds an extra dimension
        inputs = batch[:, :, :]
        targets = batch[:, :, :]  # Select the rest as targets
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.unsqueeze(-1).to(device)  # Adds an extra dimension
            inputs = batch[:, :, :]
            targets = batch[:, :, :]  # Select the rest as targets
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)  # Return average loss


def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs,  device):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"{model.__class__.__name__} Validation Loss: {val_loss:.4f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{model.__class__.__name__}.pth')
            best_val_loss = val_loss

In [246]:
# Create the models, criteria, and optimizers
model1 = PriceChangePrediction().to(device)
model2 = PriceDirectionPrediction().to(device)

criterion1 = nn.MSELoss()
criterion2 = nn.BCEWithLogitsLoss()

optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

epochs = 1

# Train and evaluate PriceChangePrediction model
train_and_evaluate(model1, train_loader, test_loader, criterion1, optimizer1, epochs, device=device)

# Train and evaluate PriceDirectionPrediction model
train_and_evaluate(model2, train_loader, test_loader, criterion2, optimizer2, epochs, device=device)

Epoch 1/1
PriceChangePrediction Validation Loss: 0.0001
Epoch 1/1
PriceDirectionPrediction Validation Loss: -0.0017


In [ ]:
test_dataset = PriceDataset('ETHUSDT', '1m', '2021-03-01', '2023-04-30')
test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_fn, shuffle=False, drop_last=True)